In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/TransformerLensOrg/TransformerLens.git``
    %pip install circuitsvis
    %pip install torchtyping
    
    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_35643/572068249.py:21: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_35643/572068249.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
from tqdm import tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional
from jaxtyping import Float, Int
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
# import circuitsvis as cv

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

torch.set_grad_enabled(False)

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

[2023-10-23 17:45:03,881] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
# load hf model
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")

In [4]:
# Disable folding norms and folding norms and biases so that intermediate value
# in between transformer blocks can be compared
bloom = HookedTransformer.from_pretrained("bloom-560m",fold_ln=False, fold_value_biases=False, center_writing_weights=False)

Loaded pretrained model bloom-560m into HookedTransformer


In [5]:
text = '''
TransformerLens lets you load in 50+ different open source language models,
and exposes the internal activations of the model to you. You can cache
any internal activation in the model, and add in functions to edit, remove
or replace these activations as the model runs.
'''
input_ids = tokenizer(text, return_tensors='pt')['input_ids']
gt_logits = model(input_ids)['logits'] # ground truth logits from hf
my_logits = bloom(input_ids)
centered_gt_logits = gt_logits - gt_logits.mean(-1, keepdim=True)
mean_diff = (my_logits.cpu() - centered_gt_logits).mean()
print("avg logits difference:", mean_diff.item())
max_diff = (my_logits.cpu() - centered_gt_logits).abs().max()
print("max logits difference:", max_diff.item())

avg logits difference: -1.5987565348041244e-05
max logits difference: 0.0006103515625


In [6]:
gt_cache = model(input_ids, output_hidden_states=True)['hidden_states']
_, my_cache = bloom.run_with_cache(input_ids)
use_loose_bound = False
pass_loose_bound = True
print("*"*5, "Matching hf and T-Lens residual stream in between transformer blocks", "*"*5)
for i in range(24):
    try:
        torch.testing.assert_close(my_cache['resid_pre',i], gt_cache[i].cuda())
    except:
        max_diff = (my_cache['resid_pre',i] - gt_cache[i].cuda()).abs().max()
        print(f"layer {i} \t not close, max difference: {max_diff}")
        use_loose_bound = True

if use_loose_bound:
    atol = rtol = 1e-3
    print("*"*5, f"\ttesting with atol={atol} and rtol={rtol}\t","*"*5)
    for i in range(24):
        try:
            torch.testing.assert_close(my_cache['resid_pre',i], gt_cache[i].cuda(), atol=atol, rtol=rtol)
        except:
            max_diff = (my_cache['resid_pre',i] - gt_cache[i].cuda()).abs().max()
            print(f"layer {i} \t not close, max difference: {max_diff}")
            pass_loose_bound = False

    if pass_loose_bound:
        print(f"All layers match with atol={atol} rtol={rtol}")
else: 
    print("All layers match")

***** Matching hf and T-Lens residual stream in between transformer blocks *****
layer 2 	 not close, max difference: 1.52587890625e-05
layer 3 	 not close, max difference: 1.8358230590820312e-05
layer 19 	 not close, max difference: 3.0517578125e-05
layer 20 	 not close, max difference: 3.0517578125e-05
layer 21 	 not close, max difference: 3.0517578125e-05
layer 22 	 not close, max difference: 6.103515625e-05
layer 23 	 not close, max difference: 6.866455078125e-05
***** 	testing with atol=0.001 and rtol=0.001	 *****
All layers match with atol=0.001 rtol=0.001


In [7]:
my_loss = bloom(input_ids, return_type='loss')
print("T-Lens next token loss:", my_loss.item())
gt_outputs = model(input_ids, labels=input_ids)
gt_loss = gt_outputs.loss
print("HF next token loss:", gt_loss.item())
print("diff in loss (abs):", (gt_loss-my_loss).abs().item())

T-Lens next token loss: 3.9514622688293457
HF next token loss: 3.951453685760498
diff in loss (abs): 8.58306884765625e-06
